<center><h3><b>Machine learning model-based early detection<br>of acute kidney injury in hypertensive patients
</b></h2></center>

##### Importing Libraries

In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import numpy as np
from scipy.stats import kstest
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from scipy.stats import ttest_ind, mannwhitneyu, chi2_contingency, fisher_exact
from xgboost import XGBClassifier
import shap
from sklearn.naive_bayes import GaussianNB
# from lime import lime_tabular
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.utils import resample
from pandas_gbq import to_gbq
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from google.cloud import bigquery
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import auth
from google.colab import data_table
from google.cloud import bigquery

## access to the project in Google Cloud
data_table.enable_dataframe_formatter()

auth.authenticate_user()
print('Authenticated')

project_id = "axial-matter-431303-b1"
client = bigquery.Client(project = project_id)

Authenticated


## Data collection

### Identifying hypertensive patients who developed acute kidney injury during their first admission

##### Finding hypertensive patients
    Definition: In this project, patients taking antihypertensive medications are classified as hypertensive patients. The mediciations referenced are based on the list provided by the Amercian Heart Association. Source: https://www.heart.org/en/health-topics/high-blood-pressure/changes-you-can-make-to-manage-high-blood-pressure/types-of-blood-pressure-medications

In [4]:

query = """
  SELECT
    adm.subject_id,
    adm.hadm_id,
    adm.admittime,
    emar.medication
  FROM
    physionet-data.mimiciv_hosp.admissions adm
  JOIN
    physionet-data.mimiciv_hosp.emar emar
    ON (adm.subject_id = emar.subject_id)
  AND (adm.hadm_id = emar.hadm_id)

  WHERE
    ## diuretics
    LOWER(emar.medication) LIKE '%furosemide%'
    OR LOWER(emar.medication) LIKE '%lasix%'
    OR LOWER(emar.medication) LIKE '%bumetanide%'
    OR LOWER(emar.medication) LIKE '%torsemide%'
    OR LOWER(emar.medication) LIKE '%demadex%'
    OR LOWER(emar.medication) LIKE '%chlorothiazide%'
    OR LOWER(emar.medication) LIKE '%amiloride%'
    OR LOWER(emar.medication) LIKE '%chlorthalidone%'
    OR LOWER(emar.medication) LIKE '%hygroton%'
    OR LOWER(emar.medication) LIKE '%hydrochlorothiazide%'
    OR LOWER(emar.medication) LIKE '%hctz%'
    OR LOWER(emar.medication) LIKE '%esidrix%'
    OR LOWER(emar.medication) LIKE '%hydrodiuril%'
    OR LOWER(emar.medication) LIKE '%indapamide%'
    OR LOWER(emar.medication) LIKE '%lozol%'
    OR LOWER(emar.medication) LIKE '%metolazone%'
    OR LOWER(emar.medication) LIKE '%zaroxolyn%'
    OR LOWER(emar.medication) LIKE '%triamterene%'
    OR LOWER(emar.medication) LIKE '%dyrenium%'

    ## beta-blockers
    OR LOWER(emar.medication) LIKE '%acebutolol%'
    OR LOWER(emar.medication) LIKE '%sectral%'
    OR LOWER(emar.medication) LIKE '%atenolol%'
    OR LOWER(emar.medication) LIKE '%tenormin%'
    OR LOWER(emar.medication) LIKE '%betaxolol%'
    OR LOWER(emar.medication) LIKE '%bisoprolol%'
    OR LOWER(emar.medication) LIKE '%carvedilol%'
    OR LOWER(emar.medication) LIKE '%coreg%'
    OR LOWER(emar.medication) LIKE '%coreg cr%'
    OR LOWER(emar.medication) LIKE '%labetalol%'
    OR LOWER(emar.medication) LIKE '%trandate%'
    OR LOWER(emar.medication) LIKE '%metoprolol succinate%'
    OR LOWER(emar.medication) LIKE '%toprol xl%'
    OR LOWER(emar.medication) LIKE '%kapspargo sprinkle%'
    OR LOWER(emar.medication) LIKE '%metoprolol tartrate%'
    OR LOWER(emar.medication) LIKE '%lopressor%'
    OR LOWER(emar.medication) LIKE '%nadolol%'
    OR LOWER(emar.medication) LIKE '%corgard%'
    OR LOWER(emar.medication) LIKE '%nebivolol%'
    OR LOWER(emar.medication) LIKE '%bystolic%'
    OR LOWER(emar.medication) LIKE '%pindolol%'
    OR LOWER(emar.medication) LIKE '%visken%'
    OR LOWER(emar.medication) LIKE '%propranolol%'
    OR LOWER(emar.medication) LIKE '%inderal%'
    OR LOWER(emar.medication) LIKE '%inderal la%'
    OR LOWER(emar.medication) LIKE '%innopran xl%'

    ## angiotensin-converting enzyme inhibitors(ACE inhibitors)
    OR LOWER(emar.medication) LIKE '%captopril%'
    OR LOWER(emar.medication) LIKE '%capoten%'
    OR LOWER(emar.medication) LIKE '%enalapril%'
    OR LOWER(emar.medication) LIKE '%vasotec%'
    OR LOWER(emar.medication) LIKE '%fosinopril%'
    OR LOWER(emar.medication) LIKE '%monopril%'
    OR LOWER(emar.medication) LIKE '%lisinopril%'
    OR LOWER(emar.medication) LIKE '%prinivil%'
    OR LOWER(emar.medication) LIKE '%zestril%'
    OR LOWER(emar.medication) LIKE '%perindopril%'
    OR LOWER(emar.medication) LIKE '%aceon%'
    OR LOWER(emar.medication) LIKE '%quinapril%'
    OR LOWER(emar.medication) LIKE '%accupril%'
    OR LOWER(emar.medication) LIKE '%ramipril%'
    OR LOWER(emar.medication) LIKE '%altace%'
    OR LOWER(emar.medication) LIKE '%trandolapril%'
    OR LOWER(emar.medication) LIKE '%mavik%'
    OR LOWER(emar.medication) LIKE '%benazepril%'
    OR LOWER(emar.medication) LIKE '%lotensin%'
    OR LOWER(emar.medication) LIKE '%moexipril%'
    OR LOWER(emar.medication) LIKE '%univasc%'

    ## angiotensin II receptor blockers (ARBs)
    OR LOWER(emar.medication) LIKE '%candesartan%'
    OR LOWER(emar.medication) LIKE '%atacand%'
    OR LOWER(emar.medication) LIKE '%losartan%'
    OR LOWER(emar.medication) LIKE '%cozaar%'
    OR LOWER(emar.medication) LIKE '%valsartan%'
    OR LOWER(emar.medication) LIKE '%diovan%'

    ## calcium channel blockers
    OR LOWER(emar.medication) LIKE '%amlodipine%'
    OR LOWER(emar.medication) LIKE '%norvasc%'
    OR LOWER(emar.medication) LIKE '%lotrel%'
    OR LOWER(emar.medication) LIKE '%diltiazem%'
    OR LOWER(emar.medication) LIKE '%cardizem cd%'
    OR LOWER(emar.medication) LIKE '%cardizem sr%'
    OR LOWER(emar.medication) LIKE '%dilacor xr%'
    OR LOWER(emar.medication) LIKE '%tiazac%'
    OR LOWER(emar.medication) LIKE '%felodipine%'
    OR LOWER(emar.medication) LIKE '%plendil%'
    OR LOWER(emar.medication) LIKE '%isradipine%'
    OR LOWER(emar.medication) LIKE '%dynacirc%'
    OR LOWER(emar.medication) LIKE '%dynacirc cr%'
    OR LOWER(emar.medication) LIKE '%nicardipine%'
    OR LOWER(emar.medication) LIKE '%cardene sr%'
    OR LOWER(emar.medication) LIKE '%nifedipine la%'
    OR LOWER(emar.medication) LIKE '%adalat cc%'
    OR LOWER(emar.medication) LIKE '%procardia xl%'
    OR LOWER(emar.medication) LIKE '%nisoldipine%'
    OR LOWER(emar.medication) LIKE '%sular%'
    OR LOWER(emar.medication) LIKE '%verapamil%'
    OR LOWER(emar.medication) LIKE '%calan sr%'
    OR LOWER(emar.medication) LIKE '%covera hs%'
    OR LOWER(emar.medication) LIKE '%isoptin sr%'
    OR LOWER(emar.medication) LIKE '%verelan%'

    ## alpha blockers
    OR LOWER(emar.medication) LIKE '%doxazosin%'
    OR LOWER(emar.medication) LIKE '%cardura%'
    OR LOWER(emar.medication) LIKE '%prazosin%'
    OR LOWER(emar.medication) LIKE '%minipress%'
    OR LOWER(emar.medication) LIKE '%terazosin hydrochloride%'
    OR LOWER(emar.medication) LIKE '%hytrin%'

    ## central alpha-2 receptor agonists and other centrally acting medications
    OR LOWER(emar.medication) LIKE '%methyldopa%'
    OR LOWER(emar.medication) LIKE '%aldomet%'
    OR LOWER(emar.medication) LIKE '%clonidine%'
    OR LOWER(emar.medication) LIKE '%catapres%'
    OR LOWER(emar.medication) LIKE '%duraclon%'
    OR LOWER(emar.medication) LIKE '%kapvay%'
    OR LOWER(emar.medication) LIKE '%nexiclon xr%'
    OR LOWER(emar.medication) LIKE '%guanfacine%'
    OR LOWER(emar.medication) LIKE '%intuniv%'
    OR LOWER(emar.medication) LIKE '%tenex%'

    ## combined alpha and beta-blockers
    OR LOWER(emar.medication) LIKE '%carvedilol%'
    OR LOWER(emar.medication) LIKE '%coreg%'
    OR LOWER(emar.medication) LIKE '%labetalol hydrochloride%'
    OR LOWER(emar.medication) LIKE '%normodyne%'
    OR LOWER(emar.medication) LIKE '%trandate%'

    ## blood vessel dilators (vasodilators)
    OR LOWER(emar.medication) LIKE '%hydralazine%'
    OR LOWER(emar.medication) LIKE '%apresoline%'
    OR LOWER(emar.medication) LIKE '%minoxidil%'
    OR LOWER(emar.medication) LIKE '%loniten%'

    AND adm.subject_id IS NOT NULL
    AND adm.hadm_id IS NOT NULL
    AND adm.admittime IS NOT NULL

  ORDER BY
    adm.subject_id, adm.hadm_id, adm.admittime
"""

query_job = client.query(query)
bp_med_recs = query_job.to_dataframe()
print(f"{len(bp_med_recs)} antihypertensive medication records")
bp_med_recs.head()

1727997 antihypertensive medication records


,subject_id,hadm_id,admittime,medication
0,10000032,22595853,2180-05-06 22:23:00,Furosemide
1,10000032,22595853,2180-05-06 22:23:00,Furosemide
2,10000032,22841357,2180-06-26 18:27:00,Furosemide
3,10000032,25742920,2180-08-05 23:44:00,Furosemide
4,10000032,25742920,2180-08-05 23:44:00,Furosemide


In [5]:
## find out the first admission information (i.e., subject_id, hadm_id)
## to these hypertensive patients

htn_patients = (

    bp_med_recs.copy()
    .assign(admittime=pd.to_datetime(bp_med_recs['admittime']))
    .sort_values(by=['subject_id', 'admittime'])

    ## keep only the first admission per patient
    .drop_duplicates(subset='subject_id', keep='first')
    [['subject_id', 'hadm_id', 'admittime']]
    .reset_index(drop=True)
)

print(f"{len(htn_patients)} hypertensive patients")
htn_patients.head()

55065 hypertensive patients


,subject_id,hadm_id,admittime
0,10000032,22595853,2180-05-06 22:23:00
1,10000764,27897940,2132-10-14 23:31:00
2,10000980,24947999,2190-11-06 20:57:00
3,10001401,21544441,2131-06-04 00:00:00
4,10001667,22672901,2173-08-22 17:16:00


##### Finding creatinine records to hypertensive patients (both in ICU module and HOSP module)

In [6]:
query = """

  ## find out all serum creatinine records in ICU module
  SELECT
    char.subject_id,
    char.hadm_id,
    dlab.label,
    char.valuenum AS value,
    char.charttime
  FROM
    physionet-data.mimiciv_icu.chartevents AS char
  INNER JOIN
    physionet-data.mimiciv_icu.d_items AS dlab
    ON char.itemid = dlab.itemid
  WHERE
    dlab.label like '%Creatinine (serum)%'

  UNION ALL

  ## find out all serum creatinine records in HOSP module
  SELECT
    le.subject_id,
    le.hadm_id,
    dlab.label,
    le.valuenum AS value,
    le.charttime
  FROM
    physionet-data.mimiciv_hosp.labevents AS le
  INNER JOIN
    physionet-data.mimiciv_hosp.d_labitems AS dlab
    ON le.itemid = dlab.itemid
  WHERE
    dlab.label LIKE '%Creatinine, Serum%'
    AND le.hadm_id IS NOT NULL

  ORDER BY
    subject_id, charttime
"""

creatinine_records = client.query(query).to_dataframe()
print(f"{len(creatinine_records)} creatinine records")
creatinine_records.head()

425647 creatinine records


,subject_id,hadm_id,label,value,charttime
0,10000032,29079034,Creatinine (serum),0.5,2180-07-23 21:45:00
1,10001217,24597018,Creatinine (serum),0.4,2157-11-21 03:16:00
2,10001217,27703517,Creatinine (serum),0.5,2157-12-20 01:45:00
3,10001725,25563031,Creatinine (serum),0.8,2110-04-11 18:02:00
4,10001725,25563031,Creatinine (serum),0.8,2110-04-12 02:59:00


In [7]:
## inner join the hypertensive patients table and the creatinine records table

htn_creatinine = (

      pd.merge(htn_patients, creatinine_records, on=['subject_id', 'hadm_id'], how='inner')
      .reset_index(drop=True)
)

print(f"{len(list(htn_creatinine['subject_id'].unique()))} hypertensive patients had {len(htn_creatinine)} creatinine records during first admission")
htn_creatinine.head()

15741 hypertensive patients had 124181 creatinine records during first admission


,subject_id,hadm_id,admittime,label,value,charttime
0,10002348,22725460,2112-11-30 22:22:00,Creatinine (serum),0.8,2112-12-01 06:36:00
1,10002348,22725460,2112-11-30 22:22:00,Creatinine (serum),0.8,2112-12-03 06:36:00
2,10002348,22725460,2112-11-30 22:22:00,Creatinine (serum),0.8,2112-12-04 06:50:00
3,10002348,22725460,2112-11-30 22:22:00,Creatinine (serum),0.9,2112-12-05 02:53:00
4,10002348,22725460,2112-11-30 22:22:00,Creatinine (serum),0.8,2112-12-06 00:17:00


##### Diagnosing acute kidney injury in hypertensive patients base on creatinine change
    Definition: According to KDIGO guidelines, one of the criteria is
    "Increase in serum creatinine by >= 0.3 mg/dL within 48 hours".

In [8]:

def creatinine_change(patient_records):

  ## create a 48-hour rolling window
  patient_records.set_index('charttime', inplace=True)
  rolling_window = pd.Series(patient_records['value']).rolling('48h')

  ## calculate the max - min within the window
  patient_records['Serum_creat_change'] = round(rolling_window.max() - rolling_window.min(), 3)

  return patient_records



## find our all patients who had a significant change in
## serum creatine >= 0.3 mg/dL within 48 hrs, the records were sourced from ICU module
aki_patients_scr = (

    htn_creatinine.copy()
    .groupby(['subject_id'], group_keys=False)
    .apply(creatinine_change)
    .assign(aki=lambda df: np.where(df['Serum_creat_change'] >= 0.3, 1, 0))
    .loc[lambda df: df['aki'] == 1]
    [['subject_id', 'hadm_id', 'admittime']]
    .drop_duplicates()
    .reset_index(drop=True)
)


print(f"{len(aki_patients_scr)} hypertensive patients who developed AKI\nduring first admission based on the serum creatinine change >= 0.3 mg/dL within 48 hours.\n\n")
aki_patients_scr.head()

5974 hypertensive patients who developed AKI
during first admission based on the serum creatinine change >= 0.3 mg/dL within 48 hours.




<ipython-input-8-d274b274f594>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(creatinine_change)


,subject_id,hadm_id,admittime
0,10002443,21329021,2183-10-17 23:20:00
1,10002495,24982426,2141-05-22 20:17:00
2,10004235,24181354,2196-02-24 14:38:00
3,10004733,27411876,2174-12-04 11:28:00
4,10004764,24817563,2168-04-11 19:16:00


##### Finding urine output records to hypertensive patients (both in ICU module and HOSP module)

In [9]:

query = """

  WITH urine_records as (

    ## find out urine output records from ICU module
    SELECT
      oe.subject_id,
      oe.hadm_id,
      dlab.label,
      oe.value AS urine_output,
      oe.charttime,
      AVG(ie.patientweight) AS mean_weight_kg

    FROM
      physionet-data.mimiciv_icu.outputevents AS oe
    JOIN
      physionet-data.mimiciv_icu.d_items AS dlab
      ON oe.itemid = dlab.itemid
    JOIN
      physionet-data.mimiciv_icu.inputevents AS ie
      ON oe.subject_id = ie.subject_id
      AND oe.hadm_id = ie.hadm_id
    WHERE
      dlab.label LIKE "Foley%"
    GROUP BY
      oe.subject_id, oe.hadm_id, dlab.label, oe.value, oe.charttime

    UNION ALL

    ## find out urine output records from HOSP module
    SELECT
      le.subject_id,
      le.hadm_id,
      dlab.label,
      le.valuenum AS urine_output,
      le.charttime,

      ## 1 pound (lb) is approximately 0.453592 kilograms (kg)
      ROUND(AVG(CAST(omr.result_value AS FLOAT64))*0.453592, 3) AS mean_weight_kg

    FROM
      physionet-data.mimiciv_hosp.labevents AS le
    JOIN
      physionet-data.mimiciv_hosp.d_labitems AS dlab
      ON le.itemid = dlab.itemid
    JOIN
      physionet-data.mimiciv_hosp.omr AS omr
      ON le.subject_id = omr.subject_id
    WHERE
      lower(dlab.label) like '%urine volume%'
      AND omr.result_name = 'Weight (Lbs)'
      AND le.hadm_id IS NOT NULL
    GROUP BY
      le.subject_id, le.hadm_id, le.charttime,
      le.valuenum, le.valueuom, dlab.label, dlab.fluid

    ORDER BY
      subject_id, charttime
  )

  SELECT
    *,
    round(urine_output / mean_weight_kg, 3) AS urine_per_kg_hr
  FROM
    urine_records
  ORDER BY
    subject_id, charttime

  """

query_job = client.query(query)
urine_output = query_job.to_dataframe()
print(f"{len(urine_output)} urine output records in based on catheterisation")
urine_output.head()

2980204 urine output records in based on catheterisation


,subject_id,hadm_id,label,urine_output,charttime,mean_weight_kg,urine_per_kg_hr
0,10000826,28289260,Urine Volume,400.0,2147-01-02 10:47:00,60.37,6.626
1,10000980,26913865,Foley,450.0,2189-06-27 09:08:00,76.20,5.906
2,10000980,26913865,Foley,600.0,2189-06-27 11:00:00,76.20,7.874
3,10000980,26913865,Foley,800.0,2189-06-27 13:00:00,76.20,10.499
4,10000980,26913865,Foley,1000.0,2189-06-27 14:00:00,76.20,13.123


In [10]:
## inner join the hypertensive patients table and the icu urine output table

htn_urine = (

      pd.merge(htn_patients, urine_output, on=['subject_id', 'hadm_id'], how='inner')
      .query('urine_per_kg_hr != 0')
      .reset_index(drop=True)
)

print(f"{len(list(htn_urine['subject_id'].unique()))} hypertensive patients had {len(htn_urine)} urine records during first admission")
htn_urine.head()

12646 hypertensive patients had 799565 urine records during first admission


,subject_id,hadm_id,admittime,label,urine_output,charttime,mean_weight_kg,urine_per_kg_hr
0,10002348,22725460,2112-11-30 22:22:00,Foley,175.0,2112-12-05 04:49:00,41.6,4.207
1,10002348,22725460,2112-11-30 22:22:00,Foley,140.0,2112-12-05 05:00:00,41.6,3.365
2,10002348,22725460,2112-11-30 22:22:00,Foley,100.0,2112-12-05 06:00:00,41.6,2.404
3,10002495,24982426,2141-05-22 20:17:00,Foley,700.0,2141-05-22 20:33:00,64.1,10.920
4,10002495,24982426,2141-05-22 20:17:00,Foley,400.0,2141-05-22 21:00:00,64.1,6.240


##### Diagnosing acute kidney injury in hypertensive patients base on urine output
    Definition: According to KDIGO guidelines, one of the criteria is
    "Urine volume < 0.5 mL/kg/hour for six hours".

In [11]:

## determine whether the current patient
## had a consecutive low urine output (>= 6 hours)
def is_low_urine(patient_records):

    aki = []
    low_urine_start = None

    for index, row in patient_records.iterrows():
        if row['urine_per_kg_hr'] < 0.5:

            if low_urine_start is None:
                low_urine_start = row['charttime']

            duration = row['charttime'] - low_urine_start

            if duration >= pd.Timedelta(hours=6):
                aki.append(row[['subject_id', 'hadm_id', 'admittime']].to_dict())
                return pd.DataFrame(aki)
        else:
            low_urine_start = None

    return pd.DataFrame(aki)


aki_patients_urine = (

    htn_urine.copy()
    .groupby(['subject_id'], group_keys=False)
    .apply(is_low_urine)
    .reset_index(drop=True)
)

print(f"{len(aki_patients_urine)} hypertensive patients who developed AKI\nduring first admission based on the urine output < 0.5 mL/kg/hour >= 6 hours.\n\n")
aki_patients_urine.head()

3433 hypertensive patients who developed AKI
during first admission based on the urine output < 0.5 mL/kg/hour >= 6 hours.




<ipython-input-11-407111d68039>:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(is_low_urine)


,subject_id,hadm_id,admittime
0,10004235,24181354,2196-02-24 14:38:00
1,10007818,22987108,2146-06-10 16:37:00
2,10010867,22429197,2147-12-30 08:40:00
3,10014610,23258342,2173-12-19 11:00:00
4,10017437,28440971,2118-04-17 00:00:00


##### Data organization
    1. Merging hypertensive acute kidney injury patients from serum creatinine records and urine output records
    2. labelling those patients with AKI and those without AKI

In [12]:
## concatenate hypertensive aki patients
## from serum creatinine table and urine output table

aki_patients = (
  pd.concat([aki_patients_scr, aki_patients_urine], ignore_index=True)
  .drop_duplicates()
  .sort_values(by=['subject_id'])
  .reset_index(drop=True)
)

print(f"{len(aki_patients)} hypertensive patients who developed AKI during first admission.")
aki_patients.head()

7079 hypertensive patients who developed AKI during first admission.


,subject_id,hadm_id,admittime
0,10002443,21329021,2183-10-17 23:20:00
1,10002495,24982426,2141-05-22 20:17:00
2,10004235,24181354,2196-02-24 14:38:00
3,10004733,27411876,2174-12-04 11:28:00
4,10004764,24817563,2168-04-11 19:16:00


In [13]:
## label hypertensive patients with acute kidney injuey as 1
## and these without this condition as 0

hypertensive_patients = (

    pd.merge(htn_patients, aki_patients, on = ['subject_id', 'hadm_id', 'admittime'], how='left', indicator=True)
    .assign(aki=lambda df:  np.where(df['_merge'] == 'both', 1, 0))
    .drop(columns=['_merge'])
)

print(f"In conclusion, {len(hypertensive_patients[hypertensive_patients['aki'] == 1])} acute kidney injury case happened among {len(hypertensive_patients)} hypertensive patients during their first admission.")
hypertensive_patients.head()


In conclusion, 7079 acute kidney injury case happened among 55065 hypertensive patients during their first admission.


,subject_id,hadm_id,admittime,aki
0,10000032,22595853,2180-05-06 22:23:00,0
1,10000764,27897940,2132-10-14 23:31:00,0
2,10000980,24947999,2190-11-06 20:57:00,0
3,10001401,21544441,2131-06-04 00:00:00,0
4,10001667,22672901,2173-08-22 17:16:00,0


In [14]:
## download the hypertensive patients table with aki label to local

from google.colab import files

hypertensive_patients.to_csv('htn_patients.csv', index=False)
files.download('htn_patients.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Data supplement (demographic and clinicial characteristics)

##### Demographic


In [15]:
query_demo = """

  WITH demo AS (
    SELECT
      patients.subject_id,
      admissions.hadm_id,
      patients.gender,
      patients.anchor_age AS age,
      ## 1 pound (lb) is approximately 0.453592 kilograms (kg)
      ROUND(AVG(CAST(omr.result_value AS FLOAT64) * 0.453592), 3) AS body_weight,
      admissions.admittime,

    FROM
      `physionet-data.mimiciv_hosp.patients` AS patients
    JOIN
      `physionet-data.mimiciv_hosp.admissions` AS admissions
      ON patients.subject_id = admissions.subject_id
    LEFT JOIN
      `physionet-data.mimiciv_hosp.omr` AS omr
      ON patients.subject_id = omr.subject_id
      AND omr.result_name = 'Weight (Lbs)'
    GROUP BY
      patients.subject_id, admissions.hadm_id, patients.gender,
      patients.anchor_age, admissions.admittime
  )

  SELECT *
  FROM
    demo
  WHERE
    gender IS NOT NULL
    AND age IS NOT NULL
    AND body_weight IS NOT NULL
  ORDER BY
    subject_id, admittime
"""

demo_df = client.query(query_demo).to_dataframe()
print(f"{len(demo_df)} records regarding to patients' demographics.")
demo_df.head()

338566 records regarding to patients' demographics.


,subject_id,hadm_id,gender,age,body_weight,admittime
0,10000032,22595853,F,52,42.231,2180-05-06 22:23:00
1,10000032,22841357,F,52,42.231,2180-06-26 18:27:00
2,10000032,29079034,F,52,42.231,2180-07-23 12:35:00
3,10000032,25742920,F,52,42.231,2180-08-05 23:44:00
4,10000084,23052089,M,72,77.111,2160-11-21 01:56:00


In [16]:
## filtering out these records relates to hypertensive patients

htn_demo_df = (
    pd.merge(hypertensive_patients, demo_df, on = ['subject_id', 'hadm_id', 'admittime'], how = 'left')
    .reset_index(drop=True)
)

htn_demo_df.head()

,subject_id,hadm_id,admittime,aki,gender,age,body_weight
0,10000032,22595853,2180-05-06 22:23:00,0,F,52,42.231
1,10000764,27897940,2132-10-14 23:31:00,0,M,86,90.496
2,10000980,24947999,2190-11-06 20:57:00,0,F,73,75.278
3,10001401,21544441,2131-06-04 00:00:00,0,F,89,75.863
4,10001667,22672901,2173-08-22 17:16:00,0,F,86,55.202


##### History of chronic diseases

In [17]:
# Obtain history of chronic diseases (hypertension, diabetes, cardiovascular disease, cerebrovascular disease, chronic pulmonary disease, liver disease, tumors, etc.)

query_chronic = """

  WITH chronic_disease as (
    SELECT
      subject_id,
      hadm_id,

      CASE WHEN lower(long_title) LIKE '%diabetes%' THEN 1 ELSE 0 END AS diabetes,

      ## Cardiovascular Disease
      CASE
        WHEN
          lower(long_title) LIKE '%heart disease%'
          OR (lower(long_title) LIKE '%heart failure%'
          AND lower(long_title) NOT LIKE '%without heart failure%')
          OR lower(long_title) LIKE '%vascular diseases%'
          OR lower(long_title) LIKE '%stroke%'
        THEN 1 ELSE 0 END AS cvd,

      ## Cerebrovascular Disease
      CASE WHEN lower(long_title) LIKE '%cerebrovascular disease%' THEN 1 ELSE 0 END AS cevd,

      ## Chronic Pulmonary Disease
      CASE
        WHEN
          lower(long_title) LIKE '%asthma%'
          OR lower(long_title) LIKE '%pulmonary fibrosis%'
        THEN 1 ELSE 0 END AS cpd,

      CASE WHEN lower(long_title) LIKE '%liver disease%' THEN 1 ELSE 0 END AS liver_disease,
      CASE WHEN lower(long_title) LIKE '%renal disease%' THEN 1 ELSE 0 END AS renal_disease,
      CASE WHEN lower(long_title) LIKE '%tumor%' THEN 1 ELSE 0 END AS tumor,
      CASE WHEN lower(long_title) LIKE '%hiv%' THEN 1 ELSE 0 END AS aids

    FROM
      `physionet-data.mimiciv_hosp.diagnoses_icd`
    JOIN
      `physionet-data.mimiciv_hosp.d_icd_diagnoses` USING (icd_code)
    WHERE
      lower(long_title) like '%diabetes%'

      OR lower(long_title) LIKE '%heart disease%'
      OR (lower(long_title) LIKE '%heart failure%'
      AND lower(long_title) NOT LIKE '%without heart failure%')
      OR lower(long_title) LIKE '%vascular diseases%'
      OR lower(long_title) LIKE '%stroke%'

      OR lower(long_title) LIKE '%cerebrovascular disease%'
      OR lower(long_title) LIKE '%asthma%'
      OR lower(long_title) LIKE '%pulmonary fibrosis%'
      OR lower(long_title) LIKE '%liver disease%'
      OR lower(long_title) LIKE '%renal disease%'
      OR lower(long_title) LIKE '%tumor%'
      OR lower(long_title) LIKE '%hiv%'
    ORDER BY
      subject_id, hadm_id
  )

  SELECT
      subject_id,
      hadm_id,
      MAX(diabetes) AS diabetes,
      MAX(cvd) AS cvd,         ## Cardiovascular disease
      MAX(cevd) AS cevd,       ## Cerebrovascular disease
      MAX(cpd) AS cpd,         ## Chronic pulmonary disease
      MAX(liver_disease) AS liver_disease,
      MAX(renal_disease) AS renal_disease,
      MAX(tumor) AS tumor,
      MAX(aids) AS aids
  FROM
      chronic_disease
  GROUP BY
      subject_id, hadm_id
  ORDER BY
      subject_id, hadm_id

"""

chronic_df = client.query(query_chronic).to_dataframe()
# print("History of chronic diseases: ", chronic_df.head())
print(f"{len(chronic_df)} records regarding to eight common chronic diseases")
chronic_df.head()

195041 records regarding to eight common chronic diseases


,subject_id,hadm_id,diabetes,cvd,cevd,cpd,liver_disease,renal_disease,tumor,aids
0,10000032,22841357,0,0,0,0,0,0,0,1
1,10000032,25742920,0,0,0,0,0,0,0,1
2,10000032,29079034,0,0,0,0,0,0,0,1
3,10000635,26134563,1,0,0,0,0,0,0,0
4,10000935,26381316,0,0,0,0,0,0,1,0


In [18]:
## filtering out these records relates to hypertensive patients

htn_chronic_df = (
    pd.merge(hypertensive_patients, chronic_df, on = ['subject_id', 'hadm_id'], how = 'left')
    .fillna(0)
    .reset_index(drop=True)
)

htn_chronic_df.head()

,subject_id,hadm_id,admittime,aki,diabetes,cvd,cevd,cpd,liver_disease,renal_disease,tumor,aids
0,10000032,22595853,2180-05-06 22:23:00,0,0,0,0,0,0,0,0,0
1,10000764,27897940,2132-10-14 23:31:00,0,0,0,0,0,0,0,0,0
2,10000980,24947999,2190-11-06 20:57:00,0,1,1,0,0,0,0,0,0
3,10001401,21544441,2131-06-04 00:00:00,0,0,0,0,0,0,0,0,0
4,10001667,22672901,2173-08-22 17:16:00,0,0,1,0,0,0,0,0,0


##### Vital signs

In [19]:
# Vital signs (heart rate, blood pressure, body temperature, etc.)
query_vitals = """

  SELECT
    subject_id,
    hadm_id,
    valuenum AS value,

    CASE
        WHEN label = 'Heart Rate' THEN 'Heart Rate'
        WHEN label = 'Non Invasive Blood Pressure systolic' THEN 'Systolic Blood Pressure'
        WHEN label = 'Non Invasive Blood Pressure diastolic' THEN 'Diastolic Blood Pressure'
        WHEN label = 'Arterial Blood Pressure mean' THEN 'Mean Arterial Pressure'
        WHEN label = 'Respiratory Rate' THEN 'Respiratory Rate'
        WHEN label = 'Temperature Celsius' THEN 'Body Temperature'
        WHEN label = 'O2 saturation pulseoxymetry' THEN 'SpO2'
        ELSE NULL
    END AS vital_sign,

    charttime

  FROM
    `physionet-data.mimiciv_icu.chartevents`
  JOIN
    `physionet-data.mimiciv_icu.d_items` USING (itemid)
  WHERE
    label IN ('Heart Rate', 'Non Invasive Blood Pressure systolic', 'Non Invasive Blood Pressure diastolic',
    'Arterial Blood Pressure mean', 'Respiratory Rate', 'Temperature Celsius', 'O2 saturation pulseoxymetry')

"""

vitals_df = client.query(query_vitals).to_dataframe()
print(f"{len(vitals_df)} records regarding to 7 common vital signs")
vitals_df.head()

29849139 records regarding to 7 common vital signs


,subject_id,hadm_id,value,vital_sign,charttime
0,18842896,24995222,111.0,Heart Rate,2148-05-11 20:40:00
1,19326831,29957742,116.0,Heart Rate,2158-07-31 14:00:00
2,14817518,21819332,115.0,Heart Rate,2138-04-10 15:00:00
3,15894036,25600846,119.0,Heart Rate,2180-11-24 17:00:00
4,10354450,28518599,121.0,Heart Rate,2163-09-25 16:00:00


In [20]:
## filtering out these records relates to hypertensive patients
## and calculate the mean value to each vital signs, that were measured during the first 24 hours of admission

htn_vitals_df = (

    pd.merge(hypertensive_patients, vitals_df, on = ['subject_id', 'hadm_id'], how = 'left')
    .dropna()
    .reset_index(drop=True)

    ## move these vital signs that were measured out of the first 24 hours of admission
    .assign(time_diff=lambda df: df['charttime'] - df['admittime'])
    .loc[lambda df: df['time_diff'] <= pd.Timedelta(hours=24)]
    .drop(columns=['charttime', 'time_diff'])

    ## calculate the mean value of each vital sign
    .groupby(['subject_id', 'hadm_id', 'admittime', 'aki', 'vital_sign'], as_index=False)['value']
    .mean()
    .round(3)

    ## transpose these vital signs to seven columns
    .pivot_table(index=['subject_id', 'hadm_id', 'admittime', 'aki'], columns='vital_sign', values='value')
    .reset_index()
)

print(f"{len(htn_vitals_df)} hypertensive patients had some vital sign records during the first 24 hours of admission")
htn_vitals_df.head()

12179 hypertensive patients had some vital sign records during the first 24 hours of admission


vital_sign,subject_id,hadm_id,admittime,aki,Body Temperature,Diastolic Blood Pressure,Heart Rate,Mean Arterial Pressure,Respiratory Rate,SpO2,Systolic Blood Pressure
0,10002348,22725460,2112-11-30 22:22:00,0,NaN,70.867,65.375,NaN,18.833,93.625,121.067
1,10002443,21329021,2183-10-17 23:20:00,1,NaN,76.808,95.107,NaN,23.607,93.000,117.423
2,10002495,24982426,2141-05-22 20:17:00,1,NaN,56.649,106.553,NaN,21.821,97.316,97.784
3,10003502,29011269,2169-08-26 16:14:00,0,NaN,51.769,56.400,NaN,21.300,95.417,126.077
4,10004113,29879900,2173-03-20 00:00:00,0,NaN,NaN,83.167,86.0,16.333,99.333,NaN


##### Laboratory findings

In [21]:
# Laboratory Findings
query_labs = """

  WITH lab_records AS (

    ## find out laboratory test in HOSP module
    SELECT
      subject_id,
      hadm_id,
      valuenum AS value,

      CASE

        WHEN label = 'Platelet Count' THEN 'Platelets'
        WHEN label = 'Urea Nitrogen' THEN 'Blood urea nitrogen'
        WHEN label = 'Creatinine, Serum' THEN 'Serum creatinine'
        WHEN label = 'INR(PT)' THEN 'International Normalized Ratio'
        WHEN label = 'Glucose' THEN 'Blood glucose'
        WHEN label = 'Anion Gap' THEN 'Anion gap'
        WHEN label = 'Bicarbonate' THEN 'Bicarbonate'
        WHEN label = 'Sodium' THEN 'Serum sodium'
        WHEN label = 'Potassium' THEN 'Serum potassium'
        WHEN label = 'Total Calcium' THEN 'Serum calcium'
        WHEN label = 'Chloride' THEN 'Serum chloride'

        ELSE label

      END AS label,

      charttime

    FROM
      `physionet-data.mimiciv_hosp.labevents`
    JOIN
      `physionet-data.mimiciv_hosp.d_labitems` USING (itemid)
    WHERE
      (label IN ('White Blood Cells', 'Hematocrit', 'Hemoglobin', 'Platelet Count',
                'Urea Nitrogen', 'INR(PT)', 'Glucose', 'Anion Gap', 'Bicarbonate',
                'Sodium', 'Potassium', 'Chloride')
      AND fluid = 'Blood')
      OR (label IN ('Creatinine, Serum')
      AND fluid = 'Urine')

    UNION ALL

    ## find out laboratory test in ICU module
    SELECT
      subject_id,
      hadm_id,
      valuenum AS value,

      CASE

        WHEN label = 'Hematocrit (serum)' THEN 'Hematocrit'
        WHEN label = 'Platelet Count' THEN 'Platelets'
        WHEN label = 'INR' THEN 'International Normalized Ratio'
        WHEN label = 'Creatinine (serum)' THEN 'Serum creatinine'
        WHEN label = 'Glucose (whole blood)' THEN 'Blood glucose'
        WHEN label = 'Sodium (serum)' THEN 'Serum sodium'
        WHEN label = 'Potassium (serum)' THEN 'Serum potassium'
        WHEN label = 'Chloride (serum)' THEN 'Serum chloride'
        WHEN label = 'Ionized Calcium' THEN 'Serum calcium'

        ELSE label

      END AS label,


      charttime,
    FROM
      physionet-data.mimiciv_icu.chartevents
    JOIN
      physionet-data.mimiciv_icu.d_items USING (itemid)
    WHERE
      label in ('Hematocrit (serum)', 'Hemoglobin', 'Platelet Count', 'INR',
              'Creatinine (serum)', 'Glucose (whole blood)', 'Anion gap',
              'Sodium (serum)', 'Potassium (serum)', 'Chloride (serum)',
              'Ionized Calcium')
  )

  SELECT
    *
  FROM
    lab_records
  WHERE
    hadm_id IS NOT NULL
  ORDER BY
    subject_id, charttime


"""

labs_df = client.query(query_labs).to_dataframe()
print(f"{len(labs_df)} records regarding to 14 laboratory findings")
labs_df.head()

25770956 records regarding to 14 laboratory findings


,subject_id,hadm_id,value,label,charttime
0,10000032,22595853,28.0,Bicarbonate,2180-05-07 05:05:00
1,10000032,22595853,99.0,Blood glucose,2180-05-07 05:05:00
2,10000032,22595853,71.0,Platelets,2180-05-07 05:05:00
3,10000032,22595853,9.0,Anion gap,2180-05-07 05:05:00
4,10000032,22595853,105.0,Serum chloride,2180-05-07 05:05:00


In [22]:
## filtering out these records relates to hypertensive patients
## and calculate the mean value to each lab test, that were measured during the first 24 hours of admission

htn_labs_df = (

    pd.merge(hypertensive_patients, labs_df, on = ['subject_id', 'hadm_id'], how = 'left')
    .dropna()
    .reset_index(drop=True)

    ## move these lab tests that were measured out of the first 24 hours of admission
    .assign(time_diff=lambda df: df['charttime'] - df['admittime'])
    .loc[lambda df: df['time_diff'] <= pd.Timedelta(hours=24)]
    .drop(columns=['charttime', 'time_diff'])

    ## calculate the mean value of each lab test
    .groupby(['subject_id', 'hadm_id', 'admittime', 'aki', 'label'], as_index=False)['value']
    .mean()
    .round(3)

    ## transpose these lab test to fourteen columns
    .pivot_table(index=['subject_id', 'hadm_id', 'admittime', 'aki'], columns='label', values='value')
    .reset_index()
)

print(f"{len(htn_labs_df)} hypertensive patients had some lab test records during the first 24 hours of admission")
htn_labs_df.head()

45013 hypertensive patients had some lab test records during the first 24 hours of admission


label,subject_id,hadm_id,admittime,aki,Anion gap,Bicarbonate,Blood glucose,Blood urea nitrogen,Hematocrit,Hemoglobin,International Normalized Ratio,Platelets,Serum calcium,Serum chloride,Serum creatinine,Serum potassium,Serum sodium,White Blood Cells
0,10000032,22595853,2180-05-06 22:23:00,0,9.0,28.0,99.0,25.0,37.600,12.7,1.5,71.0,NaN,105.0,NaN,4.5,137.0,4.2
1,10000764,27897940,2132-10-14 23:31:00,0,19.0,21.0,150.0,33.0,40.667,14.3,1.0,169.0,NaN,107.0,NaN,4.2,143.0,10.4
2,10000980,24947999,2190-11-06 20:57:00,0,18.0,22.0,273.0,29.0,26.600,9.3,1.1,158.0,NaN,106.0,NaN,3.9,142.0,6.3
3,10001667,22672901,2173-08-22 17:16:00,0,11.0,29.0,102.0,13.0,44.800,14.3,NaN,322.0,NaN,103.0,NaN,3.9,143.0,5.6
4,10001843,21728396,2131-11-09 16:05:00,0,13.0,30.0,190.0,22.0,36.800,NaN,1.7,203.0,NaN,101.0,NaN,4.1,144.0,NaN


##### Medical treatments

In [23]:
# Query medication records (inclu., Antibiotics and Analgesics)
query_treatments = """

  WITH medication_records AS (

    ## find out all antibiotics records from HOSP module
    SELECT
      adm.subject_id,
      adm.hadm_id,
      emar.charttime,
      'antibiotics' AS medication
    FROM
      `physionet-data.mimiciv_hosp.admissions` AS adm
    JOIN
      `physionet-data.mimiciv_hosp.emar` AS emar
      ON adm.subject_id = emar.subject_id
      AND adm.hadm_id = emar.hadm_id
    WHERE
      ## antibiotics (sourced from Google by search "common antibiotics")
      LOWER(emar.medication) LIKE ANY (
        '%penicillin%',
        '%cephalosporin%',
        '%tetracycline%',
        '%aminoglycoside%',
        '%oxacillin%',
        '%rifamycin%',
        '%cephalexin%',
        '%sulfamethoxazole%',
        '%trimethoprim%',
        '%fluoroquinolone%',
        '%doxycycline%',
        '%carbapenem%',
        '%amoxicillin%',
        '%clavulanic%',
        '%ciprofloxacin%',
        '%polypeptide%',
        '%macrolide%',
        '%sulfonamide%',
        '%glycopeptide%',
        '%clindamycin%',
        '%azithromycin%',
        '%erythromycin%',
        '%ampicillin%',
        '%quinolone%'
      )

    UNION ALL

    ## find out all antibiotics records from ICU module
    SELECT
      ie.subject_id,
      ie.hadm_id,
      ie.starttime as charttime,
      'antibiotics' AS medication
    FROM
      `physionet-data.mimiciv_icu.inputevents` ie
    JOIN
      `physionet-data.mimiciv_icu.d_items` USING (itemid)
    WHERE
      category = 'Antibiotics'


    UNION ALL


    ## find out all analgesics records from HOSP module
    SELECT
      adm.subject_id,
      adm.hadm_id,
      emar.charttime,
      'analgesics' AS medication
    FROM
      `physionet-data.mimiciv_hosp.admissions` AS adm
    JOIN
      `physionet-data.mimiciv_hosp.emar` AS emar
      ON adm.subject_id = emar.subject_id
      AND adm.hadm_id = emar.hadm_id
    WHERE
      ## analgesics (sourced from Google by search "common analgesics")
      LOWER(emar.medication) LIKE ANY (
        '%paracetamol%',
        '%ibuprofen%',
        '%oxycodone%',
        '%fentanyl%',
        '%indomethacin%',
        '%narcotic%',
        '%nonsteroidal%',
        '%aspirin%',
        '%opioids%',
        '%codeine%',
        '%naproxen%',
        '%meperidine%',
        '%tramadol%',
        '%diclofenac%',
        '%cyclooxygenase%',
        '%combination%',
        '%methadone%'
      )
  )

  SELECT *
  FROM
    medication_records
  WHERE hadm_id IS NOT NULL
  ORDER BY subject_id, charttime

"""
treatments_df = client.query(query_treatments).to_dataframe()
print(f"{len(treatments_df)} records regarding antibiotics and analgesics records")
treatments_df.head()

2263366 records regarding antibiotics and analgesics records


,subject_id,hadm_id,charttime,medication
0,10000032,22841357,2180-06-27 08:43:00,antibiotics
1,10000032,29079034,2180-07-24 01:21:00,analgesics
2,10000032,29079034,2180-07-24 19:32:00,analgesics
3,10000032,29079034,2180-07-25 08:25:00,analgesics
4,10000032,25742920,2180-08-06 12:19:00,analgesics


In [24]:
## filtering out these records relates to hypertensive patients
## and finding out antibiotics any analgesics medications, that were administrated during the first 24 hours of admission

htn_treatments_df = (

    pd.merge(hypertensive_patients, treatments_df, on = ['subject_id', 'hadm_id'], how = 'left')
    .reset_index(drop=True)

    ## move these records regarding the medications that were administrated out of the first 24 hours of admission
    .assign(time_diff=lambda df: df['charttime'] - df['admittime'])
    .loc[lambda df: df['time_diff'] <= pd.Timedelta(hours=24)]
    .drop(columns=['charttime', 'time_diff'])
    .drop_duplicates()
    .assign(value=1)


    ## transpose these medications to two columns (antibiotics, analgesics)
    .pivot_table(index=['subject_id', 'hadm_id', 'admittime', 'aki'], columns='medication', values='value')
    .reset_index()
    .fillna(0)

)

print(f"{len(htn_treatments_df)} hypertensive patients had antibiotics or analgesics records during the first 24 hours of admission")
htn_treatments_df.head()

36302 hypertensive patients had antibiotics or analgesics records during the first 24 hours of admission


medication,subject_id,hadm_id,admittime,aki,analgesics,antibiotics
0,10000764,27897940,2132-10-14 23:31:00,0,0.0,1.0
1,10000980,24947999,2190-11-06 20:57:00,0,1.0,0.0
2,10001843,21728396,2131-11-09 16:05:00,0,1.0,0.0
3,10001884,29675586,2130-04-08 22:06:00,0,1.0,1.0
4,10002013,24848509,2162-07-08 00:08:00,0,1.0,1.0


##### Urine output

In [25]:
## htn_urine is a table contains all urine output of hypertensive patients during their first admission
## this table was made from the above part

print(f"{len(list(htn_urine['subject_id'].unique()))} hypertensive patients had {len(htn_urine)} urine records during first admission")
htn_urine.head()

12646 hypertensive patients had 799565 urine records during first admission


,subject_id,hadm_id,admittime,label,urine_output,charttime,mean_weight_kg,urine_per_kg_hr
0,10002348,22725460,2112-11-30 22:22:00,Foley,175.0,2112-12-05 04:49:00,41.6,4.207
1,10002348,22725460,2112-11-30 22:22:00,Foley,140.0,2112-12-05 05:00:00,41.6,3.365
2,10002348,22725460,2112-11-30 22:22:00,Foley,100.0,2112-12-05 06:00:00,41.6,2.404
3,10002495,24982426,2141-05-22 20:17:00,Foley,700.0,2141-05-22 20:33:00,64.1,10.920
4,10002495,24982426,2141-05-22 20:17:00,Foley,400.0,2141-05-22 21:00:00,64.1,6.240


In [26]:
## filtering out these records relates to hypertensive patients
## and calculating the total urine output per kg weight during the first 24 hours of admission

htn_urine_df = (

    pd.merge(hypertensive_patients, htn_urine, on = ['subject_id', 'hadm_id', 'admittime'], how = 'left')
    .dropna()

    ## move these urine output that were happened out of the first 24 hours of admission
    .assign(time_diff=lambda df: df['charttime'] - df['admittime'])
    .loc[lambda df: df['time_diff'] <= pd.Timedelta(hours=24)]
    .drop(columns=['charttime', 'time_diff', 'urine_per_kg_hr'])

    ## calculate the total urine output per kg weight
    .groupby(['subject_id', 'hadm_id', 'admittime', 'aki', 'label', 'mean_weight_kg'], as_index=False)['urine_output']
    .sum()

    .assign(urine_output_kg=lambda df: df['urine_output'] / df['mean_weight_kg'])
    .round(3)
    .drop(columns=['mean_weight_kg', 'urine_output'])
    .reset_index(drop=True)
)

print(f"{len(htn_urine_df)} hypertensive patients had urine output records during the first 24 hours of admission")
htn_urine_df.head()

8356 hypertensive patients had urine output records during the first 24 hours of admission


,subject_id,hadm_id,admittime,aki,label,urine_output_kg
0,10002495,24982426,2141-05-22 20:17:00,1,Foley,54.758
1,10003502,29011269,2169-08-26 16:14:00,0,Foley,18.893
2,10004235,24181354,2196-02-24 14:38:00,1,Foley,7.323
3,10004606,29242151,2159-02-20 13:43:00,0,Foley,37.182
4,10004733,27411876,2174-12-04 11:28:00,1,Foley,32.222


##### Severity scores of illness

In [27]:
# Glasgow Coma Scale (GCS)
query_gcs = """
  SELECT
    subject_id,
    hadm_id,
    valuenum as gcs,
    charttime

  FROM
    `physionet-data.mimiciv_icu.chartevents`
  JOIN
    `physionet-data.mimiciv_icu.d_items` USING (itemid)
  WHERE
    (lower(label) like '%gcs%'
    and category = 'Neurological')
  ORDER BY
    subject_id, charttime
"""

gcs_df = client.query(query_gcs).to_dataframe()
print(f"{len(gcs_df)} GCS records")
gcs_df.head()

4882406 GCS records


,subject_id,hadm_id,gcs,charttime
0,10000032,29079034,4.0,2180-07-23 14:45:00
1,10000032,29079034,4.0,2180-07-23 14:45:00
2,10000032,29079034,6.0,2180-07-23 14:45:00
3,10000032,29079034,6.0,2180-07-23 18:22:00
4,10000032,29079034,4.0,2180-07-23 18:22:00


In [28]:
## filtering out these records relates to hypertensive patients
## and calculating the average GCS score, which were measured during the first 24 hours of admission

htn_gcs_df = (

    pd.merge(hypertensive_patients, gcs_df, on = ['subject_id', 'hadm_id'], how = 'left')
    .dropna()

    ## move these GCS score that were measured out of the first 24 hours of admission
    .assign(time_diff=lambda df: df['charttime'] - df['admittime'])
    .loc[lambda df: df['time_diff'] <= pd.Timedelta(hours=24)]
    .drop(columns=['charttime', 'time_diff'])

    ## calculate the average GCS score
    .groupby(['subject_id', 'hadm_id', 'admittime', 'aki'], as_index=False)['gcs']
    .mean()
    .round(3)
    .reset_index(drop=True)
)

print(f"{len(htn_gcs_df)} hypertensive patients had GCS score during the first 24 hours of admission")
htn_gcs_df.head()

12110 hypertensive patients had GCS score during the first 24 hours of admission


,subject_id,hadm_id,admittime,aki,gcs
0,10002348,22725460,2112-11-30 22:22:00,0,4.944
1,10002443,21329021,2183-10-17 23:20:00,1,5.000
2,10002495,24982426,2141-05-22 20:17:00,1,5.000
3,10003502,29011269,2169-08-26 16:14:00,0,3.750
4,10004113,29879900,2173-03-20 00:00:00,0,5.000


In [29]:
# SAPS II (Simplified Acute Physiology Score II)
query_saps = """
  SELECT
    subject_id,
    hadm_id,
    sapsii,
    starttime as charttime
  FROM
    `physionet-data.mimiciv_derived.sapsii`
  ORDER BY
    subject_id, hadm_id;
"""

saps_df = client.query(query_saps).to_dataframe()
print(f"{len(saps_df)} SAPS II records")
saps_df.head()

73181 SAPS II records


,subject_id,hadm_id,sapsii,charttime
0,10000032,29079034,38,2180-07-23 14:00:00
1,10000980,26913865,30,2189-06-27 08:42:00
2,10001217,24597018,13,2157-11-20 19:18:02
3,10001217,27703517,22,2157-12-19 15:42:24
4,10001725,25563031,23,2110-04-11 15:52:22


In [30]:
## filtering out these records relates to hypertensive patients
## and calculating the average SAPS II, which were measured during the first 24 hours of admission

htn_saps_df = (

    pd.merge(hypertensive_patients, saps_df, on = ['subject_id', 'hadm_id'], how = 'left')
    .dropna()

    # ## move these SAPS II score that were measured out of the first 24 hours of admission
    .assign(time_diff=lambda df: df['charttime'] - df['admittime'])
    .loc[lambda df: df['time_diff'] <= pd.Timedelta(hours=24)]
    .drop(columns=['charttime', 'time_diff'])

    # ## calculate the average SAPS II score
    .groupby(['subject_id', 'hadm_id', 'admittime', 'aki'], as_index=False)['sapsii']
    .mean()
    .round(3)
    .reset_index(drop=True)
)

print(f"{len(htn_saps_df)} hypertensive patients had SAPS II score during the first 24 hours of admission")
htn_saps_df.head()

12409 hypertensive patients had SAPS II score during the first 24 hours of admission


,subject_id,hadm_id,admittime,aki,sapsii
0,10002348,22725460,2112-11-30 22:22:00,0,51.0
1,10002443,21329021,2183-10-17 23:20:00,1,28.0
2,10002495,24982426,2141-05-22 20:17:00,1,48.0
3,10003502,29011269,2169-08-26 16:14:00,0,46.0
4,10004113,29879900,2173-03-20 00:00:00,0,15.0


##### Merging hypertensive acute kidney injury patients with their corresponding Demographic information and clinical characteristics

In [43]:
## In order to perpare the data that contains as many "important" features as possible,
## we have referred to a similar research work completed by Hu et al in 2022,
## they implemented an interpretable machine learning approach to
## predict in-hosptial mortality in critically ill patients with AKI,
## and they found that the top 4 most important variables contributing to the AKI mortality
## were GCS score, blood urea nitogen, cumulative urine output on Day 1 and age.
## source: https://www.sciencedirect.com/science/article/pii/S2001037022002173

## Based on the above insight, one of the severity score table,
## named GCS score was choosen as the based table to build our data.

## And the merging process was based on GCS_score_table.

pd.set_option('display.max_columns', 50)

hypertensive_patients_df = (

  pd.merge(htn_gcs_df, htn_saps_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .merge(htn_demo_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .merge(htn_chronic_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .merge(htn_vitals_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .merge(htn_labs_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .merge(htn_treatments_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .merge(htn_urine_df, on = ['subject_id', 'hadm_id', 'admittime', 'aki'], how = 'left')
  .drop(columns = ['label'])
)

print(len(hypertensive_patients_df))
hypertensive_patients_df.head()

12111


,subject_id,hadm_id,admittime,aki,gcs,sapsii,gender,age,body_weight,diabetes,cvd,cevd,cpd,liver_disease,renal_disease,tumor,aids,Body Temperature,Diastolic Blood Pressure,Heart Rate,Mean Arterial Pressure,Respiratory Rate,SpO2,Systolic Blood Pressure,Anion gap,Bicarbonate,Blood glucose,Blood urea nitrogen,Hematocrit,Hemoglobin,International Normalized Ratio,Platelets,Serum calcium,Serum chloride,Serum creatinine,Serum potassium,Serum sodium,White Blood Cells,analgesics,antibiotics,urine_output_kg
0,10002348,22725460,2112-11-30 22:22:00,0,4.944,51.0,F,77,42.474,0,0,0,0,0,0,0,0,NaN,70.867,65.375,NaN,18.833,93.625,121.067,12.000,23.000,127.000,26.000,39.300,13.000,1.0,273.0,NaN,107.000,0.800,4.800,142.000,4.300,NaN,NaN,NaN
1,10002443,21329021,2183-10-17 23:20:00,1,5.000,28.0,M,53,144.923,1,0,0,0,0,0,0,0,NaN,76.808,95.107,NaN,23.607,93.000,117.423,12.000,21.000,230.333,22.000,43.400,14.400,1.3,307.0,NaN,100.333,0.833,4.900,133.333,20.700,0.0,1.0,NaN
2,10002495,24982426,2141-05-22 20:17:00,1,5.000,48.0,M,81,72.575,1,1,0,0,0,0,0,0,NaN,56.649,106.553,NaN,21.821,97.316,97.784,21.333,16.667,290.000,33.667,39.725,13.767,1.3,179.0,NaN,95.667,1.533,4.033,129.667,31.533,NaN,NaN,54.758
3,10003502,29011269,2169-08-26 16:14:00,0,3.750,46.0,F,86,63.278,0,1,0,0,0,0,0,0,NaN,51.769,56.400,NaN,21.300,95.417,126.077,17.000,31.000,89.000,56.000,37.700,11.300,1.3,118.0,1.13,97.000,1.200,4.100,141.000,7.800,1.0,0.0,18.893
4,10004113,29879900,2173-03-20 00:00:00,0,5.000,15.0,NaN,<NA>,NaN,0,0,0,0,0,0,0,0,NaN,NaN,83.167,86.0,16.333,99.333,NaN,NaN,NaN,101.500,NaN,NaN,14.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN


In [44]:
## download the hypertensive patients table with aki label to local

from google.colab import files

hypertensive_patients_df.to_csv('htn_patients_info.csv', index=False)
files.download('htn_patients_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>